In [ ]:
pip install tangelo-gc

In [ ]:
pip install pyscf

In [ ]:
from tangelo import SecondQuantizedMolecule as SQMol


# cooh = ?????
o2=  """O 0.  0. 0.
     	O 1.21 0. 0. """

# 2 electrons in 2 orbitals
fo = [0,1,2,3]+[i for i in range(10,28)]
print(fo)

# Runs RHF calculation
mol_o2 = SQMol(o2, q=0, spin=2, basis='6-31g(d,p)', frozen_orbitals=fo, symmetry=True)

#Runs ROHF calculation
#mol_O2_t = SQMol(O2, q=0, spin=0, basis="6-31g(d,p)", frozen_orbitals=fo, symmetry=True)

[0, 1, 2, 3, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]


In [ ]:
print("#Energy  Symm Occ")
for i in range(8):
    print(f"{i+1:3d}{mol_o2.mo_energies[i]: 9.4f}  {mol_o2.mo_symm_labels[i]}   {int(mol_o2.mo_occ[i])}")

# Active electrons, Active orbitals
print(f"Number of active electrons: {mol_o2.n_active_electrons}")
print(f"Number of active orbtials: {mol_o2.n_active_mos}")

#Energy  Symm Occ
  1 -20.7388  A1g   2
  2 -20.7379  A1u   2
  3  -1.6497  A1g   2
  4  -1.0971  A1u   2
  5  -0.7313  A1g   2
  6  -0.7032  E1uy   2
  7  -0.7032  E1ux   2
  8  -0.2096  E1gx   1
Number of active electrons: 8
Number of active orbtials: 6


In [ ]:
print("Importing VQE, Ansatze")
from tangelo.algorithms.variational import VQESolver, BuiltInAnsatze
print("Importing FCI")
from tangelo.algorithms.classical import FCISolver

algorithm_resources = dict()

print("Solving FCI")
print(f"CASCI energy = {FCISolver(mol_o2).simulate()}")

vqe_options = {"molecule": mol_o2, "ansatz": BuiltInAnsatze.UCCSD}
print("Solving")
vqe_solver = VQESolver(vqe_options)
print("Building")
vqe_solver.build()
vqe_energy = vqe_solver.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy}")

algorithm_resources["vqe_ground_state"] = vqe_solver.get_resources()

Importing VQE, Ansatze
Importing FCI
Solving FCI
CASCI energy = -149.65514840802595
Solving
Building


TypeError: 'numpy.bool_' object does not support item assignment

In [ ]:
from tangelo.algorithms.variational import VQESolver, BuiltInAnsatze
from tangelo.algorithms.classical import FCISolver

# Dictionary of resources for each algorithm
algorithm_resources = dict()

# Ground state energy calculation with VQE, reference values with FCI
vqe_options = {"molecule": mol_o2, "ansatz": BuiltInAnsatze.UCCSD}
vqe_solver = VQESolver(vqe_options)
vqe_solver.build()
print(vqe_energy)

TypeError: 'numpy.bool_' object does not support item assignment

In [ ]:
vqe_solver.build()
vqe_energy = vqe_solver.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy}")
print(f"CASCI energy = {FCISolver(mol_o2).simulate()}")
algorithm_resources["vqe_ground_state"] = vqe_solver.get_resources()

# First excited state energy calculation with VQE, reference values with FCI
vqe_options = {"molecule": mol_o2_t, "ansatz": BuiltInAnsatze.UpCCGSD}
vqe_solver_t = VQESolver(vqe_options)
vqe_solver_t.build()
vqe_energy_t = vqe_solver_t.simulate()
print("\n Lowest Triplet state")
print(f"VQE energy = {vqe_energy_t}")
print(f"CASCI energy = {FCISolver(mol_o2_t).simulate()}")
algorithm_resources["vqe_triplet_state"] = vqe_solver_t.get_resources()

#stor ground state fo rlater
ground_vqe = vqe_energy

TypeError: 'numpy.bool_' object does not support item assignment

In [ ]:
from tangelo.toolboxes.molecular_computation.frozen_orbitals import get_orbitals_excluding_homo_lumo
from tangelo import SecondQuantizedMolecule as SQMol

o2=  """O 0.  0. 0.
     	O 1.21 0. 0. """

# freeze some
fo = [0, 1, 2, 3]+[i for i in range(10,28)]

# Runs RHF calculation
mol_o2 = SQMol(o2, q=0, spin=1, basis='6-31g(d,p)', frozen_orbitals=fo, symmetry=True)

print("  #  Energy  Symm Occ")
for i in range(10):
	print(f"{i+1:3d}{mol_o2.mo_energies[i]: 9.4f}  {mol_o2.mo_symm_labels[i]}   {int(mol_o2.mo_occ[i])}")

# Active electrons, Active orbitals
print(f"Number of active electrons: {mol_o2.n_active_electrons}")
print(f"Number of active orbtials: {mol_o2.n_active_mos}")


RuntimeError: Electron number 16 and spin 1 are not consistent
Note mol.spin = 2S = Nalpha - Nbeta, not 2S+1

In [ ]:
energy_diff = (vqe_energy_t-vqe_energy)
# convert to joules
energy_diff = -energy_diff*(-2.18*10**-18)
print("Energy difference is:", energy_diff)
h = 6.626*10**-34
v = energy_diff/h
print("frequency is:", v)
c = 299792458
w = c/v
print("wavelength is:", w)

Energy difference is: -3.695445873303071e-19
frequency is: -557718966692283.56
wavelength is: -5.375331948597829e-07


In [ ]:
#Getting next excited states
# Add initial VQE optimal circuit to the deflation circuits list
deflation_circuits = [vqe_solver.optimal_circuit.copy()]

# Calculate first and second excited states by adding optimal circuits to deflation_circuits
dict_vqe = {}
for i in range(3):
    vqe_options = {"molecule": mol_o2, "ansatz": BuiltInAnsatze.UpCCGSD,
                   "deflation_circuits": deflation_circuits, "deflation_coeff": 0.4}
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    vqe_energy = vqe_solver.simulate()
    print(f"Excited state #{i+1} \t VQE energy = {vqe_energy}")
    algorithm_resources[f"vqe_deflation_state_{i+1}"] = vqe_solver.get_resources()
    dict_vqe[i+1] =  vqe_energy
    deflation_circuits.append(vqe_solver.optimal_circuit.copy())

Excited state #1 	 VQE energy = -14.784036828631956
Excited state #2 	 VQE energy = -14.68019633282876
Excited state #3 	 VQE energy = -14.469625901062075


In [ ]:
# initiailizing dictionaries
dict_diff_vqe = {}
vqe_frequencies = {}
vqe_wavelengths = {}

# for each level, difference between states, appending to list appending to dictionary
# each corresponding label represents the nth energy state.
for i in range(1,len(dict_vqe)+1):
    list_diff_energy= []
    for j in range(1,i):
        list_diff_energy.append((dict_vqe[i] - dict_vqe[j])*(-2.18*10**-18))
    # remember that ground_vqe holds the ground state.
    list_diff_energy.append(dict_vqe[i]*(-2.18*10**-18) - ground_vqe*(-2.18*10**-18))
    dict_diff_vqe[i] = list_diff_energy
print("Our different energy differences (J) are:", dict_diff_vqe) #converting to J


# for key, value in dict_diff_vqe.items():
#     vqe_frequencies[key] = [freq * 5 for freq in value]
# print(vqe_frequencies)

def find_v_and_w(value):
    h = 6.626*10**-34
    v = energy_diff/h
    c = 299792458
    w = c/v
    return v, w

for key, value in dict_diff_vqe.items():
    for freq in value:
        vqe_frequencies.setdefault(key, []).append(find_v_and_w(freq)[0])
        vqe_wavelengths.setdefault(key, []).append(find_v_and_w(freq)[1])

print("Our corresponding frequencies (Hz) are:", vqe_frequencies)
print("Our corresponding wavelengths (m) are:", vqe_wavelengths)

Our different energy differences (J) are: {1: [-1.865721023159312e-19], 2: [-2.263722808509668e-19, -4.129443831668952e-19], 3: [-6.854158221023406e-19, -4.5904354125137385e-19, -8.719879244182685e-19]}
Our corresponding frequencies (Hz) are: {1: [53160523172714.94], 2: [53160523172714.94, 53160523172714.94], 3: [53160523172714.94, 53160523172714.94, 53160523172714.94]}
Our corresponding wavelengths (m) are: {1: [5.639381257140653e-06], 2: [5.639381257140653e-06, 5.639381257140653e-06], 3: [5.639381257140653e-06, 5.639381257140653e-06, 5.639381257140653e-06]}


In [ ]:
# cooh = ?????

# diatomic oxygen
o2_xyz = [("O", (0., 0., 0.)), ("O", (0., 0., 1.21))]

##########################

from tangelo import SecondQuantizedMolecule
o2_6311gdp = SecondQuantizedMolecule(o2_xyz, q=0, spin=1)
print(f"{o2_6311gdp.n_active_mos} active molecular orbitals")
print(f"{o2_6311gdp.n_active_electrons} active electrons")

##########################

basis_sets = [
    "STO-3G",       # Simple zeta, minimal basis.
    "3-21G",        # Double zeta.
    "6-31G",        # Double zeta with more Gaussian primitives.
    "6-31G(d,p)",   # Polarization functions (+ 5 d-orbitals for all atoms except H, +3 p-orbitals for H atoms) added.
    "6-311G(d,p)",  # Triple zeta with polarization functions.
    "6-311+G(d,p)", # Triple zeta with polarization functions and diffuse functions.
    "cc-pvqz",      # Quadruple zeta.
    "cc-pv5z"       # Quintuple zeta.
]

###
o2_6311gdp.mo_occ
###
from tangelo.toolboxes.molecular_computation.frozen_orbitals import get_orbitals_excluding_homo_lumo

# Selecting HOMO-3 to LUMO+3 orbitals.
frozen = get_orbitals_excluding_homo_lumo(o2_6311gdp, homo_minus_n=6, lumo_plus_n=5)

o2_6311gdp_frozen = SecondQuantizedMolecule(o2_xyz, q=0, spin=2,
                                            basis="6-311G(d,p)",
                                            frozen_orbitals=frozen)
print(f"{o2_6311gdp_frozen.n_active_mos} active molecular orbitals")
print(f"{o2_6311gdp_frozen.n_active_electrons} active electrons")
###

##########################

# o2_6311gdp = SecondQuantizedMolecule(o2_xyz, q=0, spin=2,
#                                      basis="6-311G(d,p)",
#                                      frozen_orbitals=None)
# print(f"{o2_6311gdp.n_active_mos} active molecular orbitals")
# print(f"{o2_6311gdp.n_active_electrons} active electrons")

# # Runs RHF calculation
# mol_O2 = SQMol(o2, q=0, spin=0, basis='6-31g(d,p)', frozen_orbitals=fo, symmetry=True)

# # Runs ROHF calculation
# mol_O2_t = SQMol(o2, q=0, spin=2, basis="6-31g(d,p)", frozen_orbitals=fo, symmetry=True)

8 active molecular orbitals
12 active electrons
34 active molecular orbitals
12 active electrons


In [ ]:
deflationcircuits = [vqe_solver.optimal_circuit.copy()]

dict_vqe = {}
for i in range(2):
    vqe_options = {"molecule": mol_Li2, "ansatz": BuiltInAnsatze.UpCCGSD,
                   "deflation_circuits": deflation_circuits, "deflation_coeff": 0.1}
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    vqe_energy = vqe_solver.simulate()
    print(f"Excited state #{i+1} \t VQE energy = {vqe_energy}")
    algorithm_resources[f"vqe_deflation_state{i+1}"] = vqe_solver.get_resources()
    dict_vqe[i+1] =  vqe_energy
    deflation_circuits.append(vqe_solver.optimal_circuit.copy())

Excited state #1 	 VQE energy = -14.66973029695983
Excited state #2 	 VQE energy = -14.683946623279379
